In [4]:
"""
En este paso, levantamos el nuevo dataset y aplicamos transformaciones, one hot enconder para las columnas categoricas y scalar para las numericas. Al terminar las transformaciones aplicamos RFE para eliminar features, el resultado de este paso se guarda como matriz esparsa en un file.
En el siguiente paso se debera levantar la X y Y de files como se indica a continaucion:

y = pd.read_csv('../desafio3/datasets/df_final.csv', parse_dates=True, na_values=['nc'], low_memory=False)
y = y['nivel_desemp_matematica']
X = scipy.sparse.load_npz('../desafio3/datasets/sparse.npz')

"""
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline, make_union, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFECV
import scipy.sparse
from sklearn.model_selection import RandomizedSearchCV


df = pd.read_csv('../desafio3/datasets/df_final.csv', parse_dates=True, na_values=['nc'], low_memory=False)

df = df.drop('Unnamed: 0', axis=1)
for i in df.columns:
    if (df[i].dtype == object) or i in ['nivel_id', 'escuela_id', 'provincia_id', 'departamento_id']:
        df[i] = pd.Categorical(df[i])

X = df.drop('nivel_desemp_matematica', axis=1)
y = df['nivel_desemp_matematica']
folds=StratifiedKFold(n_splits=5,shuffle=True, random_state=42)

class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns = 'n'):
        self.columns = columns
    
    def transform(self, X, *_):
        if isinstance(X, pd.DataFrame):
            #c para tomar columnas categoricas, cualquier otro valor para tomar continuas.
            if self.columns == 'c':
                categorical_columns = [col for col in X.columns if pd.api.types.is_categorical_dtype(df[col])]
                return pd.DataFrame(X[categorical_columns])
            else:
                scal_columns = [col for col in X.columns if (pd.api.types.is_categorical_dtype(df[col]) == False)]
                return pd.DataFrame(X[scal_columns])
        else:
            raise TypeError("Este Transformador solo funciona en DF de Pandas")
    
    def fit(self, X, *_):
        return self
cat_pipe = make_pipeline(ColumnSelector('c'),OneHotEncoder(handle_unknown='error',drop = 'first'))
num_pipe = make_pipeline(ColumnSelector('n'),RobustScaler())
union = make_union(num_pipe,
                   cat_pipe)
            
lr = LogisticRegression()
rfe = RFECV(lr,step=2000, cv=folds, n_jobs=5) 
df_union = rfe.fit_transform(union.fit_transform(X), y)
scipy.sparse.save_npz('../desafio3/datasets/sparse.npz',df_union)


In [5]:
X = scipy.sparse.load_npz('../desafio3/datasets/sparse.npz')
print(X.shape)


(277573, 7014)
